In [ ]:
import pandas as ps
from urllib.request import urlopen
import requests
url = "https://raw.githubusercontent.com/micybusy/sketch/main/charts.csv"
with urlopen(url) as f:
    content = ps.read_csv(url, encoding = "utf-8")
content["rankimportance"]=101-content["rank"]
content